In [23]:
import pandas as pd

# Read the files into DataFrames
avg_states = pd.read_fwf('avg_states.txt', header=None)
min_states = pd.read_fwf('min_states.txt', header=None)
max_states = pd.read_fwf('max_states.txt', header=None)

map = {
    0: "code",
    1: "jan",
    2: "feb",
    3: "mar",
    4: "apr",
    5: "may",
    6: "jun",
    7: "jul",
    8: "aug",
    9: "sep",
    10: "oct",
    11: "nov",
    12: "dec"
}

# Display the DataFrames
avg_states = avg_states.rename(columns=map)
min_states = min_states.rename(columns=map)
max_states = max_states.rename(columns=map)

In [24]:
avg_states.to_csv('avg_states.csv', index=False)
min_states.to_csv('min_states.csv', index=False)
max_states.to_csv('max_states.csv', index=False)

In [ ]:
# Function to convert each line to CSV
def convert_to_csv(df):
    csv_lines = [["code", "jan", "feb", "march", "april",
                  "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]]
    for index, row in df.iterrows():
        csv_lines.append(','.join(map(str, row.values[0].split())))
    return csv_lines


# Convert each dataset to CSV
avg_states_csv = convert_to_csv(avg_states)
min_states_csv = convert_to_csv(min_states)
max_states_csv = convert_to_csv(max_states)

# Display the first few lines of each CSV
avg_states

,0010021895 43.10 37.40 54.50 63.40 69.50 77.50 79.20 79.50 77.80 59.70 53.20 44.90
0,0010021896 43.50 47.70 52.50 68.00 75.90 ...
1,0010021897 41.80 51.10 60.20 62.40 69.00 ...
2,0010021898 49.00 46.10 59.20 58.80 74.10 ...
3,0010021899 43.80 40.00 55.60 61.70 76.10 ...
4,0010021900 43.60 44.40 52.90 64.20 71.40 ...
...,...
12608,3650022020 47.60 48.10 58.30 61.00 69.90 ...
12609,3650022021 44.80 42.60 56.00 60.80 68.80 ...
12610,3650022022 43.00 45.80 55.00 62.50 72.40 ...
12611,3650022023 48.10 50.30 54.80 61.60 69.90 ...
